In [ ]:
%load_ext autoreload
%autoreload 2
from src.codegen import Codebase
from src.codegen.sdk.enums import ProgrammingLanguage

# codebase = Codebase('./', programming_language=ProgrammingLanguage.PYTHON)

In [ ]:
codebase = Codebase.from_repo("sqlfluff/sqlfluff", commit="14e1a23a3166b9a645a16de96f694c77a5d4abb7")

# Agent

In [ ]:
from src.codegen.extensions.tools.semantic_edit import semantic_edit

In [ ]:
file = codebase.files[100]

In [ ]:
output = semantic_edit(
    codebase,
    file.filepath,
    '''# ... existing code ...
    def process_raw_file_for_config(self, raw_str: str):
        """Process a full raw file for inline config and update self."""
        # Scan the raw file for config commands.
        for raw_line in raw_str.splitlines():
            if raw_line.startswith("-- sqlfluff"):
                # Found a in-file config command
                self.process_inline_config(raw_line)

                # Print hello world
                print('hello, world!')
# ... existing code ...''',
)

In [ ]:
print(output["diff"])

In [ ]:
print(output["diff"])

In [ ]:
print(file.content)

## Agent

In [ ]:
agent = create_workspace_agent(codebase)

In [ ]:
problem = """TSQL - L031 incorrectly triggers "Avoid using aliases in join condition" when no join present ## Expected Behaviour Both of these queries should pass, the only difference is the addition of a table alias 'a': 1/ no alias ``` SELECT [hello] FROM mytable ``` 2/ same query with alias ``` SELECT a.[hello] FROM mytable AS a ``` ## Observed Behaviour 1/ passes 2/ fails with: L031: Avoid using aliases in join condition. But there is no join condition :-) ## Steps to Reproduce Lint queries above ## Dialect TSQL ## Version sqlfluff 0.6.9 Python 3.6.9 ## Configuration N/A"""

In [ ]:
repo = "sqlfluff/sqlfluff"
problem = """TSQL - L031 incorrectly triggers "Avoid using aliases in join condition" when no join present ## Expected Behaviour Both of these queries should pass, the only difference is the addition of a table alias 'a': 1/ no alias ``` SELECT [hello] FROM mytable ``` 2/ same query with alias ``` SELECT a.[hello] FROM mytable AS a ``` ## Observed Behaviour 1/ passes 2/ fails with: L031: Avoid using aliases in join condition. But there is no join condition :-) ## Steps to Reproduce Lint queries above ## Dialect TSQL ## Version sqlfluff 0.6.9 Python 3.6.9 ## Configuration N/A"""

codebase = Codebase.from_repo("sqlfluff/sqlfluff", commit="14e1a23a3166b9a645a16de96f694c77a5d4abb7")

result = agent.invoke(
    {
        "input": f"""
You are working on SWE Bench. I have given you the following repo: sqlfluff/sqlfluff.

Here is the problem:

{problem}

Now solve it using your tools! Godspeed 🚀
"""
    },
    config={"configurable": {"session_id": "demo-2"}},
)

In [ ]:
from src.codegen.workspace.tools.reveal_symbol import reveal_symbol

symbol = codebase.get_symbol("PyFunction")
output = reveal_symbol(symbol, collect_dependencies=False)